#Problème - Session n°2 : une variable cachée

Dans ce problème, on travaille sur un jeu de données comportant 50.000 entrées $x_i$ et des cibles $y_i$. Les entrées sont des vecteurs de taille 10 (au format torch), les cibles sont des scalaires construits à partir de cinq fonctions différentes ($f_0$, ..., $f_4$) : \

$$ \forall i, \exists k\in [\![0 \;;4]\!]  \:\: \text{tel que} \: f_k(x_i) = y_i $$

Ces fonctions sont inconnues, ainsi que l'indice $k$. Par contre, on sait que le groupe des 1000 premières cibles ont été construites à partir du même indice  $k$, de même pour les mille  suivantes, et ainsi de suite.

Le but est de parvenir à rassembler les groupes de cibles qui ont été générées avec le même indice $k$ (avec la même fonction).

In [2]:
# Example d'échantillonnage du dataset
import torch
from torch.utils.data import DataLoader

! git clone https://github.com/Zaidelmo/exam_2025_session2.git
! cp exam_2025_session2/utils/utils.py .
from utils import Problem1Dataset

dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    x_batch, y_batch, k_batch, idx_batch = batch
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    print("Batch k shape:", k_batch.shape) # indice k (pas utilisable à l'entraînement)
    print("Batch indices shape:", idx_batch.shape)
    break

Cloning into 'exam_2025_session2'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 97 (delta 11), reused 6 (delta 6), pack-reused 78 (from 2)
Receiving objects: 100% (97/97), 934.99 KiB | 2.89 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Batch input shape: torch.Size([32, 10])
Batch target shape: torch.Size([32, 1])
Batch k shape: torch.Size([32])
Batch indices shape: torch.Size([32])


**Consignes :**
- Entraîner l'architecture proposée dans la cellule suivante.
- Montrer que les vecteurs 2D de self.theta permettent de répondre
  au problème posé.
- Décrire le rôle de self.theta, du vector noise \
 et ainsi que la raison de la division par 1000 (**indices // 1000** dans le code).

Les vecteurs 2D sont des embeddings des deifférents groupes, ils permettent d'avoir une reference de similarité.
Le vecteur noise ajoute un bruit gaussian pour assurer la robustesse.
On divise par 1000 car on souhaite créer 5 clusters initialement afin de grouper les éléments générés par une meme fonction dans un meme cluster, nous possédons 50000 echantillons et donc nous devons diviser par 1000 pour obtenir nos 5 clusters.

In [5]:
class DeepMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(DeepMLP, self).__init__()
        self.theta = nn.Parameter(torch.randn(50, 2))
        self.fc1 = nn.Linear(input_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, indices):
        theta_batch = self.theta[indices // 1000, :]
        noise = torch.normal(mean=torch.zeros_like(theta_batch),
                             std=torch.ones_like(theta_batch))
        x = torch.cat([x, theta_batch + noise], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x, theta_batch

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

num_epochs = 2000
batch_size = 32        # You may adjust as needed.
learning_rate = 1e-3   # A common starting learning rate.

# Instantiate dataset and dataloader.
dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate model, loss function, and optimizer.
model = DeepMLP(input_dim=10, output_dim=1)
loss_fn = nn.MSELoss()  # Standard regression loss.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# -------------------------------
# Training Loop
# -------------------------------
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for x_batch, y_batch, L_batch, idx_batch in dataloader:
        optimizer.zero_grad()  # Reset gradients.

        # Forward pass: get predictions and the corresponding theta embeddings.
        outputs, _ = model(x_batch, idx_batch)

        # Compute loss.
        loss = loss_fn(outputs, y_batch)

        # Backward pass and optimizer step.
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * x_batch.size(0)

    epoch_loss /= len(dataset)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# -------------------------------
# Post-training: Clustering group embeddings
# -------------------------------
# We now use the learned group embeddings to cluster the groups into 5 clusters.
# Since we know that the first group (indices 0-999) is generated by a single function,
# we use its cluster assignment as an anchor.
import numpy as np
from sklearn.cluster import KMeans

# Detach and convert the learned embeddings (theta) to a NumPy array.
theta_learned = model.theta.detach().cpu().numpy()  # Shape: (50, 2)

# Cluster into 5 clusters (one per underlying function).
kmeans = KMeans(n_clusters=5, random_state=42).fit(theta_learned)
clusters = kmeans.labels_

# Identify which groups are in the same cluster as group 0.
anchor_cluster = clusters[0]
same_function_groups = np.where(clusters == anchor_cluster)[0]

print("\nLearned group embeddings (theta) clustering:")
for group_idx, cluster_id in enumerate(clusters):
    print(f"Group {group_idx:2d}: Cluster {cluster_id}")

print(f"\nGroups predicted to be generated by the same function as group 0 (anchor): {same_function_groups.tolist()}")

KeyboardInterrupt: 